In [2]:
!pip install netCDF4
import netCDF4
from netCDF4 import Dataset
import datetime as dt
from netCDF4 import date2num, num2date
import numpy as np
import pandas as pd
import cftime

In [3]:
main_path = 'C:/Users/julia/Documents/Juliana/Health Intern/GFDL-ESM4/'
scenarios = ['ssp370']
# years = [2015, 2025, 2035, 2045, 2055]
years = [2065, 2075, 2085, 2095, 2100]

In [7]:
def get_mda8(arr):
    MDA8 = []

    for la in lat:
        for lo in lon:
            listofhrs = []
            eighthr_avgs = []
            eighthr_maxima = []
            n = 0
            total = []
            site = eval('file[:,lat.tolist().index(la),lon.tolist().index(lo)]') 
            for hour in site:
                if n < 23:
                    listofhrs.append(hour)
                    n += 1
                else:
                    listofhrs.append(hour)
                    # Calculate running mean for every 8-hour period in a day in 2015
                    a = 0
                    b = 7
                    while b < len(listofhrs):
                        total = listofhrs[a:b]
                        eighthr_avgs.append(np.mean(total))
                        a += 1
                        b += 1
                    eighthr_maxima.append(max(eighthr_avgs))
                    #print(listofhrs)
                    listofhrs = []
                    n = 0
                eighthr_avgs = []
            MDA8.append(np.mean(eighthr_maxima))

    #print(MDA8)
    #print(len(MDA8))
    #print(np.mean(MDA8))
    return np.array(MDA8) 

In [11]:
 for ssp in scenarios:
    
    path = main_path + 'sfo3_AERhr_GFDL-ESM4_' + ssp + '_r1i1p1f1_gr1_206501010030-207412312330.nc'
    ncdata = Dataset(path, mode = 'r')
    
    print(ncdata.dimensions)
    lat = ncdata.variables['lat'][:]
    lon = ncdata.variables['lon'][:]

    # convert longitude format from (0, 360) to (-180, 180)
    lon_180 = ((np.array(ncdata.variables['lon'][:]) + 180) % 360) - 180
    
    # convert netcdf time to datetime
    time_num2date = num2date(ncdata.variables['time'][:],ncdata.variables['time'].units, calendar = 'gregorian')
    print(time_num2date)

    arr = np.array([time_num2date,lat,lon], ndmin=3)

    print(arr)
    print('number of dimensions :', arr.ndim)
    
    for year in years:
        
        # retrieve all the o3 forecase for the specific year
        #year_index = np.where(pd.DatetimeIndex(time_num2date).year == year)
        #print(year_index)
        #o3_year = ncdata.variables['sfo3'] # (12, 180, 288)
        #print(o3_year)
        
        # retrieve hourly o3
        hourly_o3 = ncdata.variables['sfo3']
        print(hourly_o3)
        # take annual average
        #o3_year_avg = np.mean(o3_year, axis = 0) # (180, 288)
        
        # unit conversion
        o3ppb = hourly_o3 * 10 ** 9 # (180, 288)
        
        # flatten o3 to 1-d matrix by lat[i], lon[j], pm[i, j]
        o3_avg_stacked = [o3ppb_year_avg[i, j] for i in range(0, len(ncdata.variables['lat']))
                     for j in range(0, len(ncdata.variables['lon']))]
        lat_stacked = [ncdata.variables['lat'][i] for i in range(0, len(ncdata.variables['lat']))
                     for j in range(0, len(ncdata.variables['lon']))]
        lon_stacked = [lon_180[j] for i in range(0, len(ncdata.variables['lat']))
                     for j in range(0, len(ncdata.variables['lon']))]
        
        o3data = pd.DataFrame(np.transpose([lat_stacked, lon_stacked, o3_avg_stacked]), columns = ['lat', 'lon', 'o3'])
        
        #o3_points.to_csv(main_path + 'esm4_o3_' + ssp + '_' + str(year) + '.csv')
    
    ncdata.close()
    get_mda8(o3data)

{'bnds': <class 'netCDF4._netCDF4.Dimension'>: name = 'bnds', size = 2, 'lat': <class 'netCDF4._netCDF4.Dimension'>: name = 'lat', size = 180, 'lon': <class 'netCDF4._netCDF4.Dimension'>: name = 'lon', size = 288, 'time': <class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time', size = 87600}
[datetime.datetime(2064, 11, 9, 0, 30)
 datetime.datetime(2064, 11, 9, 1, 30)
 datetime.datetime(2064, 11, 9, 2, 30) ...
 datetime.datetime(2074, 11, 6, 21, 30)
 datetime.datetime(2074, 11, 6, 22, 30)
 datetime.datetime(2074, 11, 6, 23, 30)]
[[[array([datetime.datetime(2064, 11, 9, 0, 30),
       datetime.datetime(2064, 11, 9, 1, 30),
       datetime.datetime(2064, 11, 9, 2, 30), ...,
       datetime.datetime(2074, 11, 6, 21, 30),
       datetime.datetime(2074, 11, 6, 22, 30),
       datetime.datetime(2074, 11, 6, 23, 30)], dtype=object)
   masked_array(data=[-89.5, -88.5, -87.5, -86.5, -85.5, -84.5, -83.5, -82.5,
                   -81.5, -80.5, -79.5, -78.5, -77.5, -76.5, -75.5, -74.5,
  

TypeError: unsupported operand type(s) for *: 'netCDF4._netCDF4.Variable' and 'int'